## Preprocessing  

The dataset contains the following columns:

***title:*** The title of the recipe (e.g., "Chocolate Cake").  
***ingredients:*** A list of ingredients required for the recipe (e.g., "flour", "sugar", "eggs").  
***directions:*** The step-by-step cooking instructions for preparing the recipe.  
***NER:*** Pre-labeled named entities, which can include ingredients, quantities, and other relevant information extracted from the text.  
***source:*** The source or author of the recipe.  
***link:*** The URL link to the recipe page.  
***site:*** The website or platform where the recipe is hosted (e.g., "cookbooks").  

In [ ]:
import pandas as pd

In [ ]:
#Read the dataset
df = pd.read_csv('recipes_data.csv')

In [ ]:
#Number of rows by the number of columns in the dataset
df.shape

(2231142, 7)

In [ ]:
#Return first 5 rows in the dataset
df.head()

,title,ingredients,directions,link,source,NER,site
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. evaporated milk"", ""1/2 tsp. vanilla"", ""1/2 c. broken nuts (pecans)"", ""2 Tbsp. butter or margarine"", ""3 1/2 c. bite size shredded rice biscuits""]","[""In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine."", ""Stir over medium heat until mixture bubbles all over top."", ""Boil and stir 5 minutes more. Take off heat."", ""Stir in vanilla and cereal; mix well."", ""Using 2 teaspoons, drop and shape into 30 clusters on wax paper."", ""Let stand until firm, about 30 minutes.""]",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""bite size shredded rice biscuits"", ""vanilla"", ""brown sugar"", ""nuts"", ""milk"", ""butter""]",www.cookbooks.com
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned chicken breasts"", ""1 can cream of mushroom soup"", ""1 carton sour cream""]","[""Place chipped beef on bottom of baking dish."", ""Place chicken on top of beef."", ""Mix soup and cream together; pour over chicken. Bake, uncovered, at 275\u00b0 for 3 hours.""]",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""cream of mushroom soup"", ""beef"", ""sour cream"", ""chicken breasts""]",www.cookbooks.com
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg. cream cheese, cubed"", ""1/3 c. butter, cubed"", ""1/2 tsp. garlic powder"", ""1/2 tsp. salt"", ""1/4 tsp. pepper""]","[""In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.""]",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""pepper"", ""cream cheese"", ""garlic powder"", ""butter"", ""salt""]",www.cookbooks.com
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans chicken gravy"", ""1 (10 1/2 oz.) can cream of mushroom soup"", ""1 (6 oz.) box Stove Top stuffing"", ""4 oz. shredded cheese""]","[""Boil and debone chicken."", ""Put bite size pieces in average size square casserole dish."", ""Pour gravy and cream of mushroom soup over chicken; level."", ""Make stuffing according to instructions on box (do not make too moist)."", ""Put stuffing on top of chicken and gravy; level."", ""Sprinkle shredded cheese on top and bake at 350\u00b0 for approximately 20 minutes or until golden and bubbly.""]",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken gravy"", ""cream of mushroom soup"", ""chicken"", ""shredded cheese""]",www.cookbooks.com
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker crumbs"", ""1 c. melted butter"", ""1 lb. (3 1/2 c.) powdered sugar"", ""1 large pkg. chocolate chips""]","[""Combine first four ingredients and press in 13 x 9-inch ungreased pan."", ""Melt chocolate chips and spread over mixture. Refrigerate for about 20 minutes and cut into pieces before chocolate gets hard."", ""Keep in refrigerator.""]",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""graham cracker crumbs"", ""powdered sugar"", ""peanut butter"", ""chocolate chips"", ""butter""]",www.cookbooks.com


In [ ]:
#Names of all columns in the dataset
df.columns

Index(['title', 'ingredients', 'directions', 'link', 'source', 'NER', 'site'], dtype='object')

In [ ]:
#Number of unique values for each variable
df.nunique(axis=0)

title          1312870
ingredients    2226362
directions     2211644
link           2231142
source               2
NER            2004769
site                28
dtype: int64

In [ ]:
#summarize the count, unique values, top and frequency
df.describe()

,title,ingredients,directions,link,source,NER,site
count,2231141,2231142,2231142,2231142,2231142,2231142,2231142
unique,1312870,2226362,2211644,2231142,2,2004769,28
top,Chicken Casserole,"[""1 c. peanut butter"", ""1 c. sugar"", ""1 egg""]","[""Mix all ingredients together.""]",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,[],www.cookbooks.com
freq,4099,28,274,1,1643098,573,896341


## Data Cleaning & Standardization  

Before any further processing, it is essential to clean and normalize the dataset:

**Handling Missing Values**  

Remove rows with missing title, ingredients or directions since these are essential for identifying a recipe.

**Removing Duplicates**  
Remove duplicate rows where all fields are identical, for title ingredients and directions.

**Remove unwanted columns**  
Drop link, source and site columns from the dataset as they are not required for further processing.

**Standardizing Text**  
Convert the string representation of lists to actual lists.  
Convert all text columns (title, directions, ingredients, NER) to lowercase to ensure uniformity and reduce the impact of case sensitivity.  
Combine the title, ingredients, directions into one feature on which further preprocessing can be done.
Remove special characters, extra spaces, and unwanted formatting from the combined feature column.  
Replace invalid characters or numbers with the corrected inputs(E.g. where 1 is incorrectly represented as the character l).

**Remove Noise**  
Rows having predefined NER labels that match less than 90% of the ingredients list are dropped to avoid inconsistencies in the training data and to have a more clean and accurate data.  
Remove rows that are not related to recipe data.

***Note:***
Tokenization, stop word removal, and lemmatization were not performed as part of preprocessing because we aim to preserve phrases in ingredient names for readability. For example, 'cream of mushroom soup' should remain intact.

In [ ]:
# Remove exact duplicates based on title, ingredients, and directions
df_cleaned = df.drop_duplicates(subset=['title', 'ingredients', 'directions'], keep='first')

In [ ]:
#Drop columns that won't be needed for future NER on ingredients and quantities
df_cleaned.drop(['link','source','site'], axis=1, inplace=True)

In [ ]:
#Find count of rows with null values
df_cleaned.isnull().sum()

title          1
ingredients    0
directions     0
NER            0
dtype: int64

In [ ]:
#Drop rows with null values
df_cleaned = df_cleaned.dropna()

In [ ]:
#Convert to list, change to lower case, remove rows with empty lists in NER, combine relevant features in a single column

import ast

# Convert the string representations of lists into actual lists
df_cleaned['ingredients'] = df_cleaned['ingredients'].apply(lambda x: [i.lower() for i in ast.literal_eval(x)])
df_cleaned['directions'] = df_cleaned['directions'].apply(lambda x: [i.lower() for i in ast.literal_eval(x)])
df_cleaned['NER'] = df_cleaned['NER'].apply(lambda x: [i.lower() for i in ast.literal_eval(x)])

# Remove rows with empty lists in the 'NER' column and reset the index
df_cleaned = df_cleaned[df_cleaned['NER'].apply(lambda x: x != [])].reset_index(drop=True)

# Concatenate relevant columns for NER
df_cleaned['combined'] = df_cleaned['title'] + ' ' + df_cleaned['ingredients'].astype(str) + ' ' + df_cleaned['directions'].astype(str)
df_cleaned.head()



In [ ]:
pd.set_option('display.max_colwidth',None)

#convert the combined column to lower case as it now also contains title
df_cleaned['combined'] = df_cleaned['combined'].apply(lambda x: x.lower())
df_cleaned['combined'][0]

"no-bake nut cookies ['1 c. firmly packed brown sugar', '1/2 c. evaporated milk', '1/2 tsp. vanilla', '1/2 c. broken nuts (pecans)', '2 tbsp. butter or margarine', '3 1/2 c. bite size shredded rice biscuits'] ['in a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.', 'stir over medium heat until mixture bubbles all over top.', 'boil and stir 5 minutes more. take off heat.', 'stir in vanilla and cereal; mix well.', 'using 2 teaspoons, drop and shape into 30 clusters on wax paper.', 'let stand until firm, about 30 minutes.']"

In [ ]:
#Identify and drop records that may be noisy for Named Entity recognition of ingredients
#Approach - Find the rows with less than 90% of pretrained NER labels from the ingredients list based on count of ingredients and NER labels and drop them

# Function to check if any NER label is found in the ingredient phrase
def verify_ner_labels(ingredients, ner_labels):
    incorrect_ingredients = []
    for ingredient in ingredients:
        match_found = False

        for label in ner_labels:
            # Check if the label is a substring of the ingredient phrase (case-insensitive)
            if label.lower() in ingredient.lower():
                match_found = True
                break
        if not match_found:
            incorrect_ingredients.append(ingredient)
    return incorrect_ingredients

df_filter = df_cleaned[['ingredients', 'NER']] 

# Apply the NER verification function to each row in the batch
df_filter['incorrect_ingredients'] = df_filter.apply(
    lambda row: verify_ner_labels(row['ingredients'], row['NER']), axis=1
)

# Filter rows where less than 90% of ingredients are found in NER
def is_partial_match(incorrect_ingredients, ingredients):
    # Calculate the percentage of ingredients not found in NER
    incorrect_count = len(incorrect_ingredients)
    total_count = len(ingredients)
    match_percentage = (total_count - incorrect_count) / total_count * 100
    return match_percentage < 90

# Filter rows where less than 90% of ingredients are in NER
partial_ner_df = df_filter[df_filter.apply(
    lambda row: is_partial_match(row['incorrect_ingredients'], row['ingredients']), axis=1
)]

# Check if any rows have partial NER matches and drop them from df_cleaned
if len(partial_ner_df) > 0:
    print(f"Found {len(partial_ner_df)} rows with less than 90% NER matches {len(df_filter)} rows")
    # Get the indices of rows with partial matches
    partial_ner_indices = partial_ner_df.index
    print("Sample of rows with partial NER matches:")
    partial_ner_df.head()
    # Drop these rows from the original df_cleaned
    df_cleaned = df_cleaned.drop(partial_ner_indices)
else:
    print("No rows with less than 90% NER matches")

/tmp/ipykernel_422006/26527082.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_batch['incorrect_ingredients'] = df_batch.apply(


Found 300246 rows with less than 90% NER matches 2230568 rows


In [ ]:
#Remove out of context rows that were identified during removing noisy records
df_cleaned = df_cleaned[~df_cleaned['combined'].str.contains(r'\b(scripture cake|bible cake)\b', case=False, na=False)]
len(df_cleaned)

/tmp/ipykernel_422006/194344833.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_cleaned = df_cleaned[~df_cleaned['combined'].str.contains(r'\b(scripture cake|bible cake)\b', case=False, na=False)]


1930209

In [ ]:
#Clean the dataset of special characters, punctuations, spaces where required

import re

df_cleaned['normalized_combined']= df_cleaned['combined'].apply(
    lambda x: re.sub(r'\\+[a-zA-Z]|[^a-z0-9\s./-]', ' ',  # Keep alphanumeric characters, spaces, periods, slashes, and hyphens
                       # Replace multiple spaces with a single space
                         re.sub(r'(?<!\d)\.(?!\d)', ' ',  # Remove periods that are not part of numbers
                                   re.sub(r'(?<=\d|[^\w])-', ' ',  # Remove hyphens that come after numbers
                                          re.sub(r'(\d)\s*/\s*(\d)', r'\1/\2',  # Normalize fractions by removing spaces around the slash (e.g., "1 / 2" -> "1/2")
                                                 re.sub(r'(?<!\d)\s*/\s*(?!\d)', ' ',  # Remove slashes that are not followed by a number (e.g., '/brown' -> 'brown')
                                                        str(x)  # The input data to apply the normalization
                                                       ))))).strip()  # Strip leading/trailing spaces
)
df_cleaned['normalized_combined'] = df_cleaned['normalized_combined'].str.replace(r'\s+', ' ', regex=True).str.strip()

# Print the cleaned data
df_cleaned.head()

,title,ingredients,directions,NER,combined,normalized_combined
0,No-Bake Nut Cookies,"[1 c. firmly packed brown sugar, 1/2 c. evaporated milk, 1/2 tsp. vanilla, 1/2 c. broken nuts (pecans), 2 tbsp. butter or margarine, 3 1/2 c. bite size shredded rice biscuits]","[in a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine., stir over medium heat until mixture bubbles all over top., boil and stir 5 minutes more. take off heat., stir in vanilla and cereal; mix well., using 2 teaspoons, drop and shape into 30 clusters on wax paper., let stand until firm, about 30 minutes.]","[bite size shredded rice biscuits, vanilla, brown sugar, nuts, milk, butter]","no-bake nut cookies ['1 c. firmly packed brown sugar', '1/2 c. evaporated milk', '1/2 tsp. vanilla', '1/2 c. broken nuts (pecans)', '2 tbsp. butter or margarine', '3 1/2 c. bite size shredded rice biscuits'] ['in a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.', 'stir over medium heat until mixture bubbles all over top.', 'boil and stir 5 minutes more. take off heat.', 'stir in vanilla and cereal; mix well.', 'using 2 teaspoons, drop and shape into 30 clusters on wax paper.', 'let stand until firm, about 30 minutes.']",no-bake nut cookies 1 c firmly packed brown sugar 1/2 c evaporated milk 1/2 tsp vanilla 1/2 c broken nuts pecans 2 tbsp butter or margarine 3 1/2 c bite size shredded rice biscuits in a heavy 2 quart saucepan mix brown sugar nuts evaporated milk and butter or margarine stir over medium heat until mixture bubbles all over top boil and stir 5 minutes more take off heat stir in vanilla and cereal mix well using 2 teaspoons drop and shape into 30 clusters on wax paper let stand until firm about 30 minutes
1,Jewell Ball'S Chicken,"[1 small jar chipped beef, cut up, 4 boned chicken breasts, 1 can cream of mushroom soup, 1 carton sour cream]","[place chipped beef on bottom of baking dish., place chicken on top of beef., mix soup and cream together; pour over chicken. bake, uncovered, at 275° for 3 hours.]","[cream of mushroom soup, beef, sour cream, chicken breasts]","jewell ball's chicken ['1 small jar chipped beef, cut up', '4 boned chicken breasts', '1 can cream of mushroom soup', '1 carton sour cream'] ['place chipped beef on bottom of baking dish.', 'place chicken on top of beef.', 'mix soup and cream together; pour over chicken. bake, uncovered, at 275° for 3 hours.']",jewell ball s chicken 1 small jar chipped beef cut up 4 boned chicken breasts 1 can cream of mushroom soup 1 carton sour cream place chipped beef on bottom of baking dish place chicken on top of beef mix soup and cream together pour over chicken bake uncovered at 275 for 3 hours
2,Creamy Corn,"[2 (16 oz.) pkg. frozen corn, 1 (8 oz.) pkg. cream cheese, cubed, 1/3 c. butter, cubed, 1/2 tsp. garlic powder, 1/2 tsp. salt, 1/4 tsp. pepper]","[in a slow cooker, combine all ingredients. cover and cook on low for 4 hours or until heated through and cheese is melted. stir well before serving. yields 6 servings.]","[frozen corn, pepper, cream cheese, garlic powder, butter, salt]","creamy corn ['2 (16 oz.) pkg. frozen corn', '1 (8 oz.) pkg. cream cheese, cubed', '1/3 c. butter, cubed', '1/2 tsp. garlic powder', '1/2 tsp. salt', '1/4 tsp. pepper'] ['in a slow cooker, combine all ingredients. cover and cook on low for 4 hours or until heated through and cheese is melted. stir well before serving. yields 6 servings.']",creamy corn 2 16 oz pkg frozen corn 1 8 oz pkg cream cheese cubed 1/3 c butter cubed 1/2 tsp garlic powder 1/2 tsp salt 1/4 tsp pepper in a slow cooker combine all ingredients cover and cook on low for 4 hours or until heated through and cheese is melted stir well before serving yields 6 servings
4,Reeses Cups(Candy),"[1 c. peanut butter, 3/4 c. graham cracker crumbs, 1 c. melted butter, 1 lb. (3 1/2 c.) powdered sugar, 1 large pkg. chocolate chips]","[combine first four ingredients and press in 13 x 9-inch ungreased pan., melt 

In [ ]:
#Find rows where l is part of fraction
filtered_rows = df_cleaned[df_cleaned['normalized_combined'].str.contains(r'\bl/\b', case=False, na=False)]
#Find index of rows with l from the filtered rows
matching_rows = filtered_rows[filtered_rows['normalized_combined'].str.contains('l', regex=False)].index
#Replace all l in filtered rows with 1 as these rows have incorrectly represented 1 as l
df_cleaned.loc[matching_rows, 'normalized_combined'] = df_cleaned.loc[matching_rows, 'normalized_combined'].str.replace('l', '1', regex=False)

# Replace 'l' with '1' only when it is followed by ' lb' in the entire dataset
df_cleaned['normalized_combined'] = df_cleaned['normalized_combined'].str.replace(r'(\d*)\s*l\s+lb', r'\1 1 lb', regex=True)

# Replace 'l' with 'liter' for cases where a number is followed by the letter 'l'
df_cleaned['normalized_combined'] = df_cleaned['normalized_combined'].str.replace(r'\b\d+(\.\d+)?\s+l(?!/)\b', lambda x: x.group(0).replace(' l', ' liter'), regex=True)

# Add space where an alphabet or word is followed by / so that it is not interpreted as a fraction. E.g,25 g/1/4 cup becomes 25 g / 1/4 cup
df_cleaned['normalized_combined'] = df_cleaned['normalized_combined'].apply(lambda x: re.sub(r'(?<=\w)(?<!\d)/(?=\S)', r' / ', x))

## Normalization  

Normalize all the units of measurements into as standard format. E.g., tsp, teaspoons, teaspoon should all be converted to teaspoon.  
Normalize all quantities in fractions and mixed fractions into floating point representations.  
Find all words in the lists and append to a single string.   

In [ ]:
import re
from fractions import Fraction

# Custom normalization mapping (abbreviations and different forms)
normalization_map = {
    'tsp': 'teaspoon',
    'teaspoons': 'teaspoon',
    'tbsp': 'tablespoon',
    'tablespoons': 'tablespoon',
    'oz': 'ounce',
    'ounces': 'ounce',
    'c': 'cup',
    'cup': 'cup',
    'cups': 'cup',
    'lb': 'pound',
    'lbs': 'pound',
    'pound': 'pound',
    'pounds': 'pound',
    'g': 'gram',
    'grams': 'gram',
    'kg': 'kilogram',
    'kilograms': 'kilogram',
    'cups': 'cup',
    'pkg': 'package',
    'qt': 'quart',
    'quarts': 'quart',
    'l': 'liter',
    'liters': 'liter',
    'ml': 'milliliter'
}
cnt=0
# Function to convert mixed fractions (e.g., 1 1/2) or simple fractions to decimals (e.g., 1.5)
def convert_fraction_to_decimal(quantity):
    parts = quantity.split()  # Split by space to handle mixed fractions
    if len(parts) == 2:  # Handle mixed fraction
        whole_number = int(parts[0])
        fraction = parts[1]
        try:
            return round(whole_number + float(Fraction(fraction)), 2)
        except ZeroDivisionError:
            return quantity
    elif len(parts) == 1:  # Handle simple fractions or whole numbers
        if '/' in parts[0]:  # Simple fraction
            try:
                result = round(float(Fraction(parts[0])), 2)  # Convert fraction to decimal
                return result
            except ZeroDivisionError:
                return parts[0]
       # return round(float(parts[0]), 2)  # Whole number
    return 0.0  # Fallback for any unexpected case

# Function to normalize text and convert fractions
def normalize_text(text, normalization_map):
    global cnt
    cnt+=1
    #print(cnt)
    #if(cnt==1167):
     #   print(text)
    # Regular expression pattern to match simple, mixed fractional numbers and words
    pattern = r'(\d+/\d+|\d+\s*\d*/\d+|\d*\.\d+|\d+|\S+)'

    # Find all matches in the text
    matches = re.findall(pattern, text)

    normalized_words = []
    for match in matches:
        #if(cnt==1167):
         #   print(match)
        # Check if the match is a fraction or a number
        if '/' in match:
            #print(match)
            if re.match(r'^\d+\s*\d*/\d+$', match):  # Valid fraction like 1/2, 3/4, etc.
                if re.match(r'^\d+\s+/\d+$|^/$', match):
                    pass
                else:
                    match = str(convert_fraction_to_decimal(match))
        
        # Normalize the unit using the normalization map if it's a word
        normalized_word = normalization_map.get(match.lower(), match)
        normalized_words.append(normalized_word)
    
    # Reconstruct the text from the normalized words
    return ' '.join(normalized_words)

# Example input

df_cleaned['normalized_combined'] = df_cleaned['normalized_combined'].apply(lambda x: normalize_text(x, normalization_map))
#print(normalized_text)
df_cleaned.head()


,title,ingredients,directions,NER,combined,normalized_combined
0,No-Bake Nut Cookies,"[1 c. firmly packed brown sugar, 1/2 c. evaporated milk, 1/2 tsp. vanilla, 1/2 c. broken nuts (pecans), 2 tbsp. butter or margarine, 3 1/2 c. bite size shredded rice biscuits]","[in a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine., stir over medium heat until mixture bubbles all over top., boil and stir 5 minutes more. take off heat., stir in vanilla and cereal; mix well., using 2 teaspoons, drop and shape into 30 clusters on wax paper., let stand until firm, about 30 minutes.]","[bite size shredded rice biscuits, vanilla, brown sugar, nuts, milk, butter]","no-bake nut cookies ['1 c. firmly packed brown sugar', '1/2 c. evaporated milk', '1/2 tsp. vanilla', '1/2 c. broken nuts (pecans)', '2 tbsp. butter or margarine', '3 1/2 c. bite size shredded rice biscuits'] ['in a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.', 'stir over medium heat until mixture bubbles all over top.', 'boil and stir 5 minutes more. take off heat.', 'stir in vanilla and cereal; mix well.', 'using 2 teaspoons, drop and shape into 30 clusters on wax paper.', 'let stand until firm, about 30 minutes.']",no-bake nut cookies 1 cup firmly packed brown sugar 0.5 cup evaporated milk 0.5 teaspoon vanilla 0.5 cup broken nuts pecans 2 tablespoon butter or margarine 3.5 cup bite size shredded rice biscuits in a heavy 2 quart saucepan mix brown sugar nuts evaporated milk and butter or margarine stir over medium heat until mixture bubbles all over top boil and stir 5 minutes more take off heat stir in vanilla and cereal mix well using 2 teaspoon drop and shape into 30 clusters on wax paper let stand until firm about 30 minutes
1,Jewell Ball'S Chicken,"[1 small jar chipped beef, cut up, 4 boned chicken breasts, 1 can cream of mushroom soup, 1 carton sour cream]","[place chipped beef on bottom of baking dish., place chicken on top of beef., mix soup and cream together; pour over chicken. bake, uncovered, at 275° for 3 hours.]","[cream of mushroom soup, beef, sour cream, chicken breasts]","jewell ball's chicken ['1 small jar chipped beef, cut up', '4 boned chicken breasts', '1 can cream of mushroom soup', '1 carton sour cream'] ['place chipped beef on bottom of baking dish.', 'place chicken on top of beef.', 'mix soup and cream together; pour over chicken. bake, uncovered, at 275° for 3 hours.']",jewell ball s chicken 1 small jar chipped beef cut up 4 boned chicken breasts 1 can cream of mushroom soup 1 carton sour cream place chipped beef on bottom of baking dish place chicken on top of beef mix soup and cream together pour over chicken bake uncovered at 275 for 3 hours
2,Creamy Corn,"[2 (16 oz.) pkg. frozen corn, 1 (8 oz.) pkg. cream cheese, cubed, 1/3 c. butter, cubed, 1/2 tsp. garlic powder, 1/2 tsp. salt, 1/4 tsp. pepper]","[in a slow cooker, combine all ingredients. cover and cook on low for 4 hours or until heated through and cheese is melted. stir well before serving. yields 6 servings.]","[frozen corn, pepper, cream cheese, garlic powder, butter, salt]","creamy corn ['2 (16 oz.) pkg. frozen corn', '1 (8 oz.) pkg. cream cheese, cubed', '1/3 c. butter, cubed', '1/2 tsp. garlic powder', '1/2 tsp. salt', '1/4 tsp. pepper'] ['in a slow cooker, combine all ingredients. cover and cook on low for 4 hours or until heated through and cheese is melted. stir well before serving. yields 6 servings.']",creamy corn 2 16 ounce package frozen corn 1 8 ounce package cream cheese cubed 0.33 cup butter cubed 0.5 teaspoon garlic powder 0.5 teaspoon salt 0.25 teaspoon pepper in a slow cooker combine all ingredients cover and cook on low for 4 hours or until heated through and cheese is melted stir well before serving yields 6 servings
4,Reeses Cups(Candy),"[1 c. peanut butter, 3/4 c. graham cracker crumbs, 1 c. melted butter, 1 lb. (3 1/2 c.) powdered sugar, 1 large pkg. chocolate chips]","[combine first four ingredien

In [ ]:
# Function to clean each item in the list
def clean_item(item):
    # Remove unwanted symbols but preserve alphabets, numbers, hyphens, and spaces
    return ''.join(e for e in item if e.isalnum() or e in ['-', ' '])

# Apply the cleaning function to each list in the 'NER' column
df_cleaned['NER'] = df_cleaned['NER'].apply(lambda x: [clean_item(item) for item in x])

# Print the cleaned DataFrame
df_cleaned['NER']

0                                                                                                                                                                                                                                                [bite size shredded rice biscuits, vanilla, brown sugar, nuts, milk, butter]
1                                                                                                                                                                                                                                                                 [cream of mushroom soup, beef, sour cream, chicken breasts]
2                                                                                                                                                                                                                                                            [frozen corn, pepper, cream cheese, garlic powder, butter, salt]
4                                             

In [ ]:
#Drop columns not needed for future training of NER model and keep only the normalized_combined feature and NER labelled data
df_cleaned.drop(['title', 'directions', 'combined', 'ingredients'], axis=1, inplace=True)
df_cleaned.head()

,NER,normalized_combined
0,"[bite size shredded rice biscuits, vanilla, brown sugar, nuts, milk, butter]",no-bake nut cookies 1 cup firmly packed brown sugar 0.5 cup evaporated milk 0.5 teaspoon vanilla 0.5 cup broken nuts pecans 2 tablespoon butter or margarine 3.5 cup bite size shredded rice biscuits in a heavy 2 quart saucepan mix brown sugar nuts evaporated milk and butter or margarine stir over medium heat until mixture bubbles all over top boil and stir 5 minutes more take off heat stir in vanilla and cereal mix well using 2 teaspoon drop and shape into 30 clusters on wax paper let stand until firm about 30 minutes
1,"[cream of mushroom soup, beef, sour cream, chicken breasts]",jewell ball s chicken 1 small jar chipped beef cut up 4 boned chicken breasts 1 can cream of mushroom soup 1 carton sour cream place chipped beef on bottom of baking dish place chicken on top of beef mix soup and cream together pour over chicken bake uncovered at 275 for 3 hours
2,"[frozen corn, pepper, cream cheese, garlic powder, butter, salt]",creamy corn 2 16 ounce package frozen corn 1 8 ounce package cream cheese cubed 0.33 cup butter cubed 0.5 teaspoon garlic powder 0.5 teaspoon salt 0.25 teaspoon pepper in a slow cooker combine all ingredients cover and cook on low for 4 hours or until heated through and cheese is melted stir well before serving yields 6 servings
4,"[graham cracker crumbs, powdered sugar, peanut butter, chocolate chips, butter]",reeses cup candy 1 cup peanut butter 0.75 cup graham cracker crumbs 1 cup melted butter 1 pound 3.5 cup powdered sugar 1 large package chocolate chips combine first four ingredients and press in 13 x 9 inch ungreased pan melt chocolate chips and spread over mixture refrigerate for about 20 minutes and cut into pieces before chocolate gets hard keep in refrigerator
5,"[sour cream, bacon, pepper, extra lean ground beef, cheddar cheese, green onion, baking potatoes, milk, butter, salt]",cheeseburger potato soup 6 baking potatoes 1 pound of extra lean ground beef 0.67 cup butter or margarine 6 cup milk 0.75 teaspoon salt 0.5 teaspoon pepper 1.5 cup 6 ounce shredded cheddar cheese divided 12 sliced bacon cooked crumbled and divided 4 green onion chopped and divided 1 8 ounce carton sour cream optional wash potatoes prick several times with a fork microwave them with a wet paper towel covering the potatoes on high for 6 8 minutes the potatoes should be soft ready to eat let them cool enough to handle cut in half lengthwise scoop out pulp and reserve discard shells brown ground beef until done drain any grease from the meat set aside when done meat will be added later melt butter in a large kettle over low heat add flour stirring until smooth cook 1 minute stirring constantly gradually add milk cook over medium heat stirring constantly until thickened and bubbly stir in potato ground beef salt pepper 1 cup of cheese 2 tablespoon of green onion and 0.5 cup of bacon cook until heated do not boil stir in sour cream if desired cook until heated do not boil sprinkle with remaining cheese bacon and green onions
...,...,...
2230563,"[chocolate hazelnut spread, marshmallows, hazelnuts, tortillas, butter]",sunny s fake crepes 0.5 cup chocolate hazelnut spread recommended nutella 4 8 inch round whole-wheat tortillas room temperature 4 tablespoon butter 1.33 cup miniature marshmallows 0.5 cup finely chopped toasted hazelnuts spread hazelnut spread on 1 side of each tortilla in a large pan over medium heat add 2 tablespoon of butter and let it melt until bubbly add 2 of the tortillas with the chocolate hazelnut spread side facing up using an imaginary line add 0.33 cup of marshmallows down 1 half of each tortilla making sure to keep them in a single layer allow the tortilla to slightly bubble on the pan and check underneath for a golden toasted color fold the tortilla over to form a half circle and slightly press down to seal cook until marshmallows start to melt but don t lose their shape transfer them to a cutting 

In [ ]:
#Write the standardized and normalized, preprocessed dataset to a csv file to be used further for Named Entity Recognition of recipe ingredients and their quantities.
df_cleaned.to_csv('preprocessed_labelled_data.csv',index=False)